In [ ]:
SPLITS = 5

In [ ]:
from keras.callbacks import EarlyStopping
from sklearn.model_selection import StratifiedShuffleSplit
import statistics
import time
from jet_ml.helpers import hms_string



# Bootstrap
boot = StratifiedShuffleSplit(n_splits=SPLITS, test_size=0.1, 
                                random_state=42)

# Track progress
mean_benchmark = []
epochs_needed = []
num = 0
split_results=[]
times_took=[]

# Loop through samples
for train, test in boot.split(x,y_raw):
    start_time = time.time()
    num+=1

    # Split train and test
    x_train = x[train]
    y_train = y[train]
    x_test = x[test]
    y_test = y[test]

    # Construct neural network
    import jet_ml.models.test_net as test_net
    input_shape=x[0].shape
    output_shape=y.shape[1]
    activation='softmax'
    model=test_net.build_model(input_shape,output_shape,activation)
    model=test_net.compile_model(model)
    

    # Train on the bootstrap sample
    batch_size=128
    # epochs=5
    # monitor='val_loss' #'val_accuracy' or 'val_loss'
    monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, 
        patience=100, verbose=0, mode='auto', restore_best_weights=True)
    model, history=test_net.train_model(model,
                                        x_train,y_train, x_test,y_test, 
                                        epochs, batch_size, monitor,
                                        fold=num)
    
    epochs = monitor.stopped_epoch
    epochs_needed.append(epochs)
    
    
  
    # # Measure this bootstrap's log loss
    # from jet_ml.evaluation import save_training_history
    # save_training_history(history=history,fold=num)

    # from jet_ml.evaluation import plot_training_history
    # plot_training_history(history=history,fold=num)

    from jet_ml.evaluation import get_accuracy
    pred, score=get_accuracy(model,x_test=x_test,y_test=y_test)
    # Predict on the out of boot (validation)

    # pred = model.predict(x_test)
    # y_compare = np.argmax(y_test,axis=1) # For log loss calculation
    # score = metrics.log_loss(y_compare, pred)
    mean_benchmark.append(score)
    # m1 = statistics.mean(mean_benchmark)
    # m2 = statistics.mean(epochs_needed)
    # mdev = statistics.pstdev(mean_benchmark)
    
    # Record this iteration
    time_took = time.time() - start_time
    times_took.append(time_took)
    # split_results.append({'split':num,
    #                       'accuracy_score':score,
    #                       'mean_score':m1,
    #                       'stdev':mdev,
    #                       'epochs':epochs,
    #                       'mean_epochs':m2,
    #                       'time':time_took})
    # print(f"#{num}: score={score:.6f}, mean score={m1:.6f}," +\
    #       f"stdev={mdev:.6f}, epochs={epochs}, mean epochs={int(m2)}," +\
    #       f" time={hms_string(time_took)}")
    print(f"#{num}: score={score:.6f}, mean score={0:.6f}," +\
          f"stdev={0:.6f}, epochs={epochs}, mean epochs={int(0)}," +\
          f" time={hms_string(time_took)}")
from jet_ml.evaluation import save_bootstraping_history
save_bootstraping_history(mean_benchmark,epochs_needed,times_took,splits=SPLITS)